# .s2p Reader

In [7]:
import os
import numpy as np
import skrf as skrf
import cmath
import math as m

#Calcula los parámetros S en una frecuencia dada para cada archivo .s2p en las carpetas especificadas.
def calcular_parametros_s(frecuencia, carpetas):
    s_parameters = {}
    
    # Convertir frecuencia de GHz a Hz
    frecuencia *= 1e9                                  
    
    for carpeta in carpetas:
        archivos_s2p = [f for f in os.listdir(carpeta) if f.endswith('.s2p')]
        for archivo_s2p in archivos_s2p:
            ruta_archivo = os.path.join(carpeta, archivo_s2p)
            red = skrf.network.Network(ruta_archivo)
            
            # Encontrar la frecuencia más cercana en el archivo .s2p
            indice_frecuencia = np.argmin(np.abs(red.f - frecuencia))
                        
            # Acceder a los parámetros S en la frecuencia encontrada
            if archivo_s2p not in s_parameters:
                s_parameters[archivo_s2p] = red.s[indice_frecuencia]
    
    return s_parameters

ModuleNotFoundError: No module named 'numpy'

### Cálculo de Delta $\Delta$

In [221]:
def calculo_delta(parametros_s):
    dts= {}
    dts_abs= {}
    for archivo, parametros in parametros_s.items():
        s11= parametros[0, 0]
        s12= parametros[0, 1]
        s21= parametros[1, 0]
        s22= parametros[1, 1]

        # delta = (parametros[0, 0] * parametros[1, 1]) - (parametros[0, 1] * parametros[1, 0])
        dt= s11*s22 - s12*s21
        dts[archivo]= dt
        dts_abs[archivo]= abs(dt)

    return dts_abs

### Cálculo de Factor de Rollet $K$

In [222]:
def calcular_k(s_parametros, deltas):
    k = {}
    k_incond = {}
    for archivo, parametros in s_parametros.items():
        s11 = parametros[0, 0]
        s12 = parametros[0, 1]
        s21 = parametros[1, 0]
        s22 = parametros[1, 1]
        abs_delta = abs(deltas[archivo])
        
        # Calcular el parámetro K
        K= (1-abs(s11)**2 - abs(s22)**2 + (abs_delta)**2)/(2*abs(s12*s21))
        
        k[archivo]= K
        if K > 1:
            # Guardar la polarización si k es mayor que 1
            k_incond[archivo] = {'k': K}
    
    return k_incond


### Cálculo de $\Gamma_{in}, \Gamma_{out}$

### Diseño de microtiras

In [224]:
# def microtiras(Zo):
#     er = 4.3
#     A = Zo/60*np.sqrt((er+1)/2) + ((er-1)/(er+1))*(0.23 + (0.11/er))
#     B = 377*np.pi/(2*Zo*np.sqrt(er))

#     print("A: ", A, "B: ", B)


#     WH = (8*np.e**A)/(np.e**(2*A)-2)
#     print("WH: ", WH)

#     WH = 2*np.pi*(B-1-np.ln(2*B-1) + (er-1)/(2*er)*(np.ln(B-1) + 0.39*0.61/er))
#     print("WH: ", WH)

### Ejemplo de uso

In [ ]:
frecuencia = 1.6                # GHz
Zo         = 50                 # Impedancia  
carpetas   = ["Datasheets/BFP640"]    # Rutas a las carpetas que contienen los archivos .s2p
ic_filter  = 40                 # Corriente para filtrar polaricaciones

s_parameters = calcular_parametros_s(frecuencia, carpetas)
deltas= calculo_delta(s_parameters)
#k, k_incond= calcular_k(s_parameters, deltas)
k_incond= calcular_k(s_parameters, deltas)
impedancias  = calcular_impedancias(Zo, s_parameters, k_incond, ic_filter, frecuencia)


## Prints 

1) Todos los parámetros S para la frecuencia establecida (nuestro caso $1.6 \space GHz$)

In [227]:
# Parámetros S de todas las polarizaciones con frecuencia 1.6 GHz.
#print(s_parameters)

# Deltas calculados
#print(deltas)

# K que son incondicionalmente estables.
#print(k_incond)